# Retrieval and Generation with Bedrock Foundational Models

## Overview  
This notebook demonstrates how to perform retrieval-augmented generation (RAG) using Amazon Bedrock's foundational models. It covers retrieving relevant documents from a knowledge base and generating responses based on the retrieved context.

In [ ]:
import json
with open("variables.json", "r") as f:
    variables = json.load(f)

variables

In [ ]:
# Knowledge Base ID - Choose from different chunking strategies (Fixed, Hierarchical, or Semantic)
kb_id = variables["kbFixedChunk"] 

# Bedrock Model ARN - Using Amazon Nova Lite for inference
model_id = f"arn:aws:bedrock:us-west-2:{variables['accountNumber']}:inference-profile/us.amazon.nova-lite-v1:0"

# Number of relevant documents to retrieve for RAG
number_of_results = 5

# Configuration for text generation - Controls output length, randomness, and diversity
generation_configuration = {
    'inferenceConfig': {
        'textInferenceConfig': {
            'maxTokens': 123,  # Maximum number of tokens in the generated response
            'stopSequences': [],  # List of sequences that indicate stopping points
            'temperature': 0.2,  # Controls randomness (lower values = more deterministic output)
            'topP': 0.5  # Controls diversity of output by considering top P probability mass
        }
    },
}

In [ ]:
import boto3

# Initialize the Bedrock Agent Runtime client
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime", region_name=variables["regionName"])

# Define the query to search relevant knowledge base documents and generate an answer
# query = "What are three sub-tasks in question answering over knowledge bases?"
query = "What were the third-person view games?"

# Perform retrieval-augmented generation (RAG) using the knowledge base
response = bedrock_agent_runtime.retrieve_and_generate(
    input={
        "text": query  # User query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,  # ID of the knowledge base used for retrieval
            "modelArn": model_id,  # Bedrock model ARN for text generation
            "generationConfiguration": generation_configuration,  # Model configuration parameters
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults": number_of_results  # Number of relevant documents to fetch
                } 
            }
        }
    }
)

# Display the generated response
print('----------------- Answer ---------------------')
print(response['output']['text'], end='\n' * 2)

# Display the full response including citations for retrieved documents
print('----------------- Citations ------------------')
print(json.dumps(response, indent=2))
